In [1]:
import pandas as pd
from datetime import datetime, timedelta
import mlflow.pyfunc
import json

# --- Leer el dataset completo con histórico ---
df = pd.read_parquet("../data/processed/full_dataset.parquet")

# --- Definir la fecha objetivo ---
tomorrow = datetime.today() + timedelta(days=1)
fecha_pred = tomorrow.date()

# --- Asegurarse que la columna Fecha esté en formato datetime ---
df["Fecha"] = pd.to_datetime(df["Fecha"])

# --- Obtener las combinaciones únicas de dimensiones ---
dim_cols = ["Estado", "Ciudad", "Tipo", "Canal"]
combinations = df[dim_cols].drop_duplicates()

# --- Calcular Precio_lag1 y Precio_mean7 por combinación ---
records = []

for _, row in combinations.iterrows():
    mask = (
        (df["Estado"] == row["Estado"]) &
        (df["Ciudad"] == row["Ciudad"]) &
        (df["Tipo"] == row["Tipo"]) &
        (df["Canal"] == row["Canal"])
    )
    df_sub = df[mask].sort_values("Fecha")

    # Filtro hasta el día anterior a la predicción
    df_sub = df_sub[df_sub["Fecha"] < pd.Timestamp(fecha_pred)]

    if len(df_sub) == 0:
        continue  # no hay datos históricos

    precio_lag1 = df_sub.iloc[-1]["Precio"]
    precio_mean7 = df_sub.tail(7)["Precio"].mean()

    record = {
        "Estado": row["Estado"],
        "Ciudad": row["Ciudad"],
        "Tipo": row["Tipo"],
        "Canal": row["Canal"],
        "día": tomorrow.day,
        "mes": tomorrow.month,
        "año": tomorrow.year,
        "dia_semana": str(tomorrow.weekday()),  # como string para DictVectorizer
        "Precio_lag1": precio_lag1,
        "Precio_mean7": precio_mean7
    }
    records.append(record)

# --- Construir DataFrame final de entrada ---
df_pred = pd.DataFrame(records)

# --- Cargar modelo desde JSON y ruta S3 ---
with open("deployment/promoted/daily_model.json", "r") as f:
    meta = json.load(f)

model_id = meta["artifact_uri"].split("/")[-1]
model_s3_path = f"s3://mlflow-models-milk-price-dev/2/models/{model_id}/artifacts/"
model = mlflow.pyfunc.load_model(model_s3_path)

# --- Hacer predicciones ---
df_pred["Precio_sugerido"] = model.predict(df_pred.to_dict(orient="records"))

# --- Guardar predicciones ---
output_path = f"reports/predicciones_{fecha_pred}.csv"
df_pred.to_csv(output_path, index=False)

print(f"✅ Archivo guardado: {output_path}")


/home/maxkaizo/.local/share/virtualenvs/milk_price_prediction-mju8LJeN/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Archivo guardado: reports/predicciones_2025-08-02.csv
